In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import pickle
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import random
np.random.seed(0)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_df = pd.read_csv('/content/drive/MyDrive/ISR/dataset/2019-Oct.csv')


In [ ]:
# data_df.drop('event_time', axis=1, inplace=True)
# data_df.drop('user_session', axis=1, inplace=True)

# data_df.isna().sum()

data_df.dropna(inplace=True)
data_df.shape

(26560620, 9)

In [ ]:
# dataset with no nil value entries - considered as our original dataset
data_df.to_pickle('/content/drive/MyDrive/ISR/dataset/df')


In [ ]:
display(data_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
8,2019-10-01 00:00:10 UTC,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
...,...,...,...,...,...,...,...,...,...
42448757,2019-10-31 23:59:58 UTC,view,1004870,2053013555631882655,electronics.smartphone,samsung,275.25,518956209,6764041a-9285-4869-8a32-a79adf31d212
42448758,2019-10-31 23:59:58 UTC,view,2702331,2053013563911439225,appliances.kitchen.refrigerators,lg,527.43,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4
42448759,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448761,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd


In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/df', 'rb') as f:
  data_df = pickle.load(f)

In [ ]:
def gnn_transform(data_df):
    unique_UserID = data_df['user_id'].unique()
    unique_product_id = data_df['product_id'].unique()
    unique_category_id = data_df['category_id'].unique()
    unique_event_id = data_df['event_type'].unique()
    
    j = 0
    user_old2new_id_dict = dict()
    for u in unique_UserID:
        user_old2new_id_dict[u] = j
        j += 1
    j = 0
    productId_old2new_id_dict = dict()
    for i in unique_product_id:
        productId_old2new_id_dict[i] = j
        j += 1
    j = 0
    categoryId_old2new_id_dict = dict()
    for i in unique_category_id:
        categoryId_old2new_id_dict[i] = j
        j += 1
    j = 0
    eventId_old2new_id_dict = dict()
    for i in unique_event_id:
        eventId_old2new_id_dict[i] = j
        j += 1
    user_list = data_df['user_id'].values
    productId_list = data_df['product_id'].values
    categoryId_list = data_df['category_id'].values
    eventId_list = data_df['event_type'].values

    for j in range(len(data_df)):
        user_list[j] = user_old2new_id_dict[user_list[j]]
        productId_list[j] = productId_old2new_id_dict[productId_list[j]]
        categoryId_list[j] = categoryId_old2new_id_dict[categoryId_list[j]]
        eventId_list[j] = eventId_old2new_id_dict[eventId_list[j]]
    data_df['user_id'] = user_list
    data_df['product_id'] = productId_list
    data_df['category_id'] = categoryId_list
    data_df['event_type'] = eventId_list
    
    return data_df, user_old2new_id_dict, productId_old2new_id_dict, categoryId_old2new_id_dict, eventId_old2new_id_dict

In [ ]:
data_df['event_type'].unique()

array(['view', 'purchase', 'cart'], dtype=object)

In [ ]:
# initial trsnforamtion for filtering
data_df, user_old2new_id_dict, productId_old2new_id_dict, categoryId_old2new_id_dict, eventId_old2new_id_dict = gnn_transform(data_df)

In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/user_old2new_id_dict', 'wb') as f:
    pickle.dump(user_old2new_id_dict, f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/productId_old2new_id_dict', 'wb') as f:
    pickle.dump(productId_old2new_id_dict, f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/categoryId_old2new_id_dict', 'wb') as f:
    pickle.dump(categoryId_old2new_id_dict, f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/eventId_old2new_id_dict', 'wb') as f:
    pickle.dump(eventId_old2new_id_dict, f)

In [ ]:
eventId_old2new_id_dict

{'view': 0, 'purchase': 1, 'cart': 2}

In [ ]:
data_df = data_df[(data_df['user_id']>=0) & (data_df['user_id']<5000)]
data_df = data_df[(data_df['category_id']>=0) & (data_df['category_id']<100)]
data_df = data_df[(data_df['product_id']>=0) & (data_df['product_id']<1000)]
display(data_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,0,0,0,appliances.environment.water_heater,aqua,33.20,0,9333dfbd-b87a-4708-9857-6336556b0fcc
3,2019-10-01 00:00:01 UTC,0,1,1,computers.notebook,lenovo,251.74,1,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,0,2,2,electronics.smartphone,apple,1081.98,2,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,0,3,3,computers.desktop,pulser,908.62,3,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
8,2019-10-01 00:00:10 UTC,0,4,4,apparel.shoes.keds,baden,102.71,4,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
...,...,...,...,...,...,...,...,...,...
42444659,2019-10-31 23:25:33 UTC,0,37,2,electronics.smartphone,samsung,717.39,2185,c27e18bb-4f9e-46bb-9a00-bb122cd32c96
42444931,2019-10-31 23:28:15 UTC,0,331,2,electronics.smartphone,apple,1013.86,2139,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445113,2019-10-31 23:29:51 UTC,0,331,2,electronics.smartphone,apple,1013.86,2139,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445150,2019-10-31 23:30:11 UTC,0,972,2,electronics.smartphone,apple,1090.89,2139,bb7cae5a-ce49-4226-b268-b77ea51e8614


In [ ]:
# trasnforming again for re-indexing the gnn dataset after filtering
data_df, user_old2new_id_dict, productId_old2new_id_dict, categoryId_old2new_id_dict, eventId_old2new_id_dict = gnn_transform(data_df)
display(data_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,0,0,0,appliances.environment.water_heater,aqua,33.20,0,9333dfbd-b87a-4708-9857-6336556b0fcc
3,2019-10-01 00:00:01 UTC,0,1,1,computers.notebook,lenovo,251.74,1,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,0,2,2,electronics.smartphone,apple,1081.98,2,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,0,3,3,computers.desktop,pulser,908.62,3,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
8,2019-10-01 00:00:10 UTC,0,4,4,apparel.shoes.keds,baden,102.71,4,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
...,...,...,...,...,...,...,...,...,...
42444659,2019-10-31 23:25:33 UTC,0,37,2,electronics.smartphone,samsung,717.39,1770,c27e18bb-4f9e-46bb-9a00-bb122cd32c96
42444931,2019-10-31 23:28:15 UTC,0,331,2,electronics.smartphone,apple,1013.86,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445113,2019-10-31 23:29:51 UTC,0,331,2,electronics.smartphone,apple,1013.86,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445150,2019-10-31 23:30:11 UTC,0,950,2,electronics.smartphone,apple,1090.89,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614


In [ ]:
data_df.to_pickle('/content/drive/MyDrive/ISR/dataset/gnn/GNN_df')

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/user_old2new_id_dict', 'wb') as f:
    pickle.dump(user_old2new_id_dict, f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/productId_old2new_id_dict', 'wb') as f:
    pickle.dump(productId_old2new_id_dict, f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/categoryId_old2new_id_dict', 'wb') as f:
    pickle.dump(categoryId_old2new_id_dict, f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/eventId_old2new_id_dict', 'wb') as f:
    pickle.dump(eventId_old2new_id_dict, f)

In [ ]:
# Start directly from here for item-item code

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import pickle
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import random
np.random.seed(0)
import json

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/gnn/GNN_df', 'rb') as f:
  gnn_data_df = pickle.load(f)

In [ ]:
display(gnn_data_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,0,0,0,appliances.environment.water_heater,aqua,33.20,0,9333dfbd-b87a-4708-9857-6336556b0fcc
3,2019-10-01 00:00:01 UTC,0,1,1,computers.notebook,lenovo,251.74,1,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,0,2,2,electronics.smartphone,apple,1081.98,2,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,0,3,3,computers.desktop,pulser,908.62,3,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
8,2019-10-01 00:00:10 UTC,0,4,4,apparel.shoes.keds,baden,102.71,4,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
...,...,...,...,...,...,...,...,...,...
42444659,2019-10-31 23:25:33 UTC,0,37,2,electronics.smartphone,samsung,717.39,1770,c27e18bb-4f9e-46bb-9a00-bb122cd32c96
42444931,2019-10-31 23:28:15 UTC,0,331,2,electronics.smartphone,apple,1013.86,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445113,2019-10-31 23:29:51 UTC,0,331,2,electronics.smartphone,apple,1013.86,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445150,2019-10-31 23:30:11 UTC,0,950,2,electronics.smartphone,apple,1090.89,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614


In [ ]:
# purchase_df = gnn_data_df.loc[:, ["event_type", "user_id"]]
purchase_df = gnn_data_df[gnn_data_df["event_type"]==1]
purchase_df = purchase_df.drop(['event_time', 'user_session'], axis=1)
purchase_df.drop_duplicates(inplace=True)
display(purchase_df[purchase_df["user_id"]==55])

,event_type,product_id,category_id,category_code,brand,price,user_id
162,1,48,2,electronics.smartphone,samsung,130.76,55
1249966,1,48,2,electronics.smartphone,samsung,130.25,55
3179981,1,48,2,electronics.smartphone,samsung,132.15,55
4503689,1,48,2,electronics.smartphone,samsung,132.05,55
6319170,1,48,2,electronics.smartphone,samsung,131.79,55
11218119,1,48,2,electronics.smartphone,samsung,130.49,55
13203956,1,97,2,electronics.smartphone,samsung,150.31,55
14373749,1,614,2,electronics.smartphone,samsung,145.41,55
15511729,1,48,2,electronics.smartphone,samsung,130.48,55
17515483,1,97,2,electronics.smartphone,samsung,145.13,55


In [ ]:
def transform(data_df):
    unique_UserID = data_df['user_id'].unique()
    unique_product_id = data_df['product_id'].unique()
    unique_category_id = data_df['category_id'].unique()
    unique_event_id = data_df['event_type'].unique()
    
    j = 0
    user_old2new_id_dict = dict()
    for u in unique_UserID:
        user_old2new_id_dict[u] = j
        j += 1
    j = 0
    productId_old2new_id_dict = dict()
    for i in unique_product_id:
        productId_old2new_id_dict[i] = j
        j += 1
    j = 0
    categoryId_old2new_id_dict = dict()
    for i in unique_category_id:
        categoryId_old2new_id_dict[i] = j
        j += 1
    j = 0
    eventId_old2new_id_dict = dict()
    for i in unique_event_id:
        eventId_old2new_id_dict[i] = j
        j += 1
    user_list = data_df['user_id'].values
    productId_list = data_df['product_id'].values
    categoryId_list = data_df['category_id'].values
    eventId_list = data_df['event_type'].values

    for j in range(len(data_df)):
        user_list[j] = user_old2new_id_dict[user_list[j]]
        productId_list[j] = productId_old2new_id_dict[productId_list[j]]
        categoryId_list[j] = categoryId_old2new_id_dict[categoryId_list[j]]
        eventId_list[j] = eventId_old2new_id_dict[eventId_list[j]]
    data_df['user_id'] = user_list
    data_df['product_id'] = productId_list
    data_df['category_id'] = categoryId_list
    data_df['event_type'] = eventId_list
    
    return data_df

In [ ]:
data_df = data_df[(data_df['user_id']>=0) & (data_df['user_id']<5000)]
data_df = data_df[(data_df['category_id']>=0) & (data_df['category_id']<100)]
data_df = data_df[(data_df['product_id']>=0) & (data_df['product_id']<1000)]
data_df = data_df[(data_df['event_type'] == 0)]

data_df.drop('event_time', axis=1, inplace=True)
data_df.drop('user_session', axis=1, inplace=True)

# data_df = transform(data_df)

In [ ]:
display(data_df)

,event_type,product_id,category_id,category_code,brand,price,user_id
1,1,0,0,appliances.environment.water_heater,aqua,33.20,0
3,1,1,1,computers.notebook,lenovo,251.74,1
4,1,2,2,electronics.smartphone,apple,1081.98,2
5,1,3,3,computers.desktop,pulser,908.62,3
8,1,4,4,apparel.shoes.keds,baden,102.71,4
...,...,...,...,...,...,...,...
42444659,1,37,2,electronics.smartphone,samsung,717.39,1770
42444931,1,331,2,electronics.smartphone,apple,1013.86,1743
42445113,1,331,2,electronics.smartphone,apple,1013.86,1743
42445150,1,950,2,electronics.smartphone,apple,1090.89,1743


In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/productId_old2new_id_dict', 'rb') as f:
    old_productId_dict = pickle.load(f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/user_old2new_id_dict', 'rb') as f:
    old_user_old2new_id_dict = pickle.load(f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/old_mappings/categoryId_old2new_id_dict', 'rb') as f:
    old_categoryId_old2new_id_dict = pickle.load(f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/productId_old2new_id_dict', 'rb') as f:
    new_productId_dict = pickle.load(f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/user_old2new_id_dict', 'rb') as f:
    new_user_old2new_id_dict = pickle.load(f)

with open('/content/drive/MyDrive/ISR/dataset/gnn/new_mappings/categoryId_old2new_id_dict', 'rb') as f:
    new_categoryId_old2new_id_dict = pickle.load(f)

# old_productId_dict
# new_productId_dict

In [ ]:
def reindex(old_dict, new_dict):
  merged_dict = defaultdict(int)
  for key, value in old_dict.items():
    if value in new_dict:
      merged_dict[new_dict[value]] = key
  return merged_dict

In [ ]:
product_idx = reindex(old_productId_dict, new_productId_dict)
user_idx = reindex(old_user_old2new_id_dict, new_user_old2new_id_dict)
catergory_idx = reindex(old_categoryId_old2new_id_dict, new_categoryId_old2new_id_dict)

with open('/content/drive/MyDrive/ISR/dataset/indexing/user', 'wb') as f:
    pickle.dump(user_idx, f)

with open('/content/drive/MyDrive/ISR/dataset/indexing/product', 'wb') as f:
    pickle.dump(product_idx, f)

with open('/content/drive/MyDrive/ISR/dataset/indexing/category', 'wb') as f:
    pickle.dump(catergory_idx, f)

In [ ]:
data_df.to_pickle('/content/drive/MyDrive/ISR/dataset/compressed_df') 

In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/compressed_df', 'rb') as f:
    data_df = pickle.load(f)

In [ ]:
num_user = len(data_df['user_id'].unique())
num_prod = len(data_df['product_id'].unique())
print(num_user, num_prod)
print(data_df.shape)

4396 973
(72783, 7)


In [ ]:
# hashing categories
cat_code = data_df['category_code']
categories = set()
for cat in cat_code:
  temp = cat.split('.')
  categories.update(temp)

In [ ]:
# user_prod_db - cat are keys and (user_id, prod_id) are values
# cat_prod_db - cat are keys and prod_id are values
user_prod_db = defaultdict(set)
cat_prod_db = defaultdict(set)
for index, row in data_df.iterrows():
  temp = row['category_code'].split('.')
  for key in temp:
    user_prod_db[key].add((row['user_id'], row['product_id']))
    cat_prod_db[key].add(row['product_id'])

# user_db - user_id are keys and prod_id are values
user_db = defaultdict(set)
for index, row in data_df.iterrows():
  temp = row['user_id']
  user_db[temp].add(row['product_id'])

In [ ]:
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

num_user = len(data_df['user_id'].unique())
num_prod = len(data_df['product_id'].unique())
num_category = len(data_df['category_id'].unique())
print(num_user, num_prod)

4396 973


In [ ]:
train_df['user_id'].values.max()

4395

In [ ]:
train_mat = coo_matrix((train_df['event_type'].values, (train_df['user_id'].values, train_df['product_id'].values)), shape=(num_user, num_prod)).astype(float).toarray()
test_mat = coo_matrix((test_df['event_type'].values, (test_df['user_id'].values, test_df['product_id'].values)), shape=(num_user, num_prod)).astype(float).toarray()

train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)

In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/train_mat', 'wb') as f:
    pickle.dump(train_mat, f)

with open('/content/drive/MyDrive/ISR/dataset/test_mat', 'wb') as f:
    pickle.dump(test_mat, f)

In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/train_mat', 'rb') as f:
    train_mat = pickle.load(f)
with open('/content/drive/MyDrive/ISR/dataset/test_mat', 'rb') as f:
    test_mat = pickle.load(f)

In [ ]:
#proucts wise cosine similarity and prod popularity matrix
prod_sim = cosine_similarity(train_mat.transpose())

popularity_matrix = np.sum(train_mat, axis=0)
popularity_matrix = np.argsort(popularity_matrix)[::-1]

In [ ]:
def boolean_retrieval(q_tokens):
  temp = []
  if q_tokens:
    ans = cat_prod_db[q_tokens[0]]
    for word in q_tokens[1:]:
        ans = ans.intersection(cat_prod_db[word])
  temp = ans

  df = pd.DataFrame(temp, columns=['product_id'])
  return df

def visitedInBooleanRet(df, userID):
    viewedList = list(user_db[userID])
    df = df[df['product_id'].isin(viewedList)]
    return df

def unVisitedInBooleanRet(df, userID):
    viewedList = list(user_db[userID])
    df = df[~df['product_id'].isin(viewedList)]
    return df

In [ ]:
with open('/content/drive/MyDrive/ISR/dataset/indexing/product', 'rb') as f:
    prod_reindex_dict = pickle.load(f)

with open('/content/drive/MyDrive/ISR/dataset/indexing/category', 'rb') as f:
    category_reindex_dict = pickle.load(f)

In [ ]:
def retrieve(product_id, user_id):
  details = data_df.loc[(data_df['product_id'] == product_id)].iloc[0]
  details = pd.DataFrame([(details['product_id'], details['category_id'], details['category_code'], details['price'], details['brand'])], index=[0], columns=['product_id', 'category_id', 'category_code', 'price', 'brand'])
  # details['category_code'] = details['category_code'].astype(str).str.split('.')
  prod_ids = details['product_id']
  cat_ids = details['category_id']

  for i in range(len(prod_ids)):
    prod_ids[i] = prod_reindex_dict[prod_ids[i]]
    cat_ids[i] = category_reindex_dict[cat_ids[i]]

  details['product_id'] = prod_ids
  details['category_id'] = cat_ids
  return details
def json_conv(user_id, result):
  ans = pd.DataFrame(columns=['product_id', 'category_id', 'category_code'])
  for i in result:
    temp = retrieve(i, user_id)
    ans = pd.concat([ans, temp], axis=0)
    ans['user_id'] = user_id
  return ans



In [ ]:
# query is array of input words
def recommender(query, userId):
  booleanRes = boolean_retrieval(query)
  alreadyVisitedRes = visitedInBooleanRet(booleanRes, userId)
  unVisitedRes = unVisitedInBooleanRet(booleanRes, userId)

  booleanProds, alreadyVisitedProds, unVisitedProds = booleanRes['product_id'].values, alreadyVisitedRes['product_id'].values, unVisitedRes['product_id'].values

  visitedCount = len(alreadyVisitedProds)
  unVisitedCount = len(unVisitedProds)
  booleanResCount = len(booleanProds)

  results = []

  # item-item CF
  if visitedCount != 0:
    booleanResSimilairty = np.zeros((train_mat.shape[1], train_mat.shape[1]))
    unVisitedProds = np.sort(unVisitedProds)

    for i in range(unVisitedCount):
      for j in range(visitedCount):
        booleanResSimilairty[unVisitedProds[i]][alreadyVisitedProds[j]] = prod_sim[unVisitedProds[i]][alreadyVisitedProds[j]]

    booleanResSimilairty = booleanResSimilairty[np.ix_(unVisitedProds, alreadyVisitedProds)]

    preferences = np.sum(booleanResSimilairty, axis=1)
    preferences = [element/visitedCount for element in preferences]
    scores = dict(zip(unVisitedProds, preferences))
    sorted_scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
    non_zero_scores = {k: v for k, v in sorted_scores.items() if v != 0.0}

    final_preferences =non_zero_scores.keys()
    results = list(final_preferences)
  # Popular Products
  else: 
    popular_results = popularity_matrix[np.in1d(popularity_matrix, booleanProds)]
    results = popular_results
  
  json_res = json_conv(userId, results)

  return json_res

In [ ]:
results = recommender(['computers'], 3)
# results = results.to_json(orient='records')
# # s = json.loads(results)
display(results)


In [ ]:
with open('/content/drive/MyDrive/ISR/purchase_history', 'rb') as f:
    purchase_history = pickle.load(f)

with open('/content/drive/MyDrive/ISR/gnn_rec', 'rb') as f:
    gnn_recommendations = pickle.load(f)

In [ ]:
def gnn_retrieve(product_id, user_id):
  details = gnn_data_df.loc[(gnn_data_df['product_id'] == product_id)].iloc[0]
  details = pd.DataFrame([(details['product_id'], details['category_id'], details['category_code'], details['price'], details['brand'])], index=[0], columns=['product_id', 'category_id', 'category_code', 'price', 'brand'])
  prod_ids = details['product_id']
  cat_ids = details['category_id']

  for i in range(len(prod_ids)):
    prod_ids[i] = prod_reindex_dict[prod_ids[i]]
    cat_ids[i] = category_reindex_dict[cat_ids[i]]

  details['product_id'] = prod_ids
  details['category_id'] = cat_ids
  return details

def gnn_json_conv(user_id, result):
  ans = pd.DataFrame(columns=['product_id', 'category_id', 'category_code']) 
  for i in result:
    temp = gnn_retrieve(i, user_id)
    ans = pd.concat([ans, temp], axis=0)
    ans['user_id'] = user_id
  return ans

In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!ngrok config add-authtoken 2P7RF3ZlNRzWd2Ls7SK6GgYvuwE_3LMLD3iimRhxTuvvUbMbD
!pip install -U flask-cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19879 sha256=b652b7dfcd6b79a93e0a2a918b2f7ecc5156258ad35f09a728fb28944e0d8fff
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, request, jsonify
import json
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from flask_cors import cross_origin

app = Flask(__name__)
CORS(app, resources={r"/api/*": {"origins": "*"}})
run_with_ngrok(app)

@app.route('/rec', methods=['POST'])
@cross_origin()
def get_recommendation():
    data = request.get_json()
    query = data['query'].split()
    userId = data['userId']
    print(query, userId)
    ans = recommender(query, int(userId))
    print(ans)
    json_res = ans.to_json(orient='records')
    return json.loads(json_res)

@app.route('/history', methods=['POST'])
@cross_origin()
def get_purchase_history():
    data = request.get_json()
    userId = data['userId']
    ans = gnn_json_conv(int(userId), purchase_history[int(userId)])
    json_res = ans.to_json(orient='records')
    return json.loads(json_res)

@app.route('/gnn', methods=['POST'])
@cross_origin()
def get_gnn_recommendations():
    data = request.get_json()
    userId = data['userId']
    print(userId)
    ans = gnn_json_conv(int(userId), gnn_recommendations[int(userId)])
    json_res = ans.to_json(orient='records')
    return json.loads(json_res)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b829-34-67-223-108.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:52:11] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:52:12] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:53:18] "OPTIONS /gnn HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:53:18] "OPTIONS /gnn HTTP/1.1" 200 -
<ipython-input-27-36849f2435f1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_ids[i] = prod_reindex_dict[prod_ids[i]]
<ipython-input-27-36849f2435f1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_ids[i] = category_reindex_dict[cat_ids[i]]
<ipython-input-27-36849f2435f1>:8: SettingWithCopyW

1402
1402


...

1402


INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:56:51] "OPTIONS /rec HTTP/1.1" 200 -
<ipython-input-16-5fee57a14482>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_ids[i] = prod_reindex_dict[prod_ids[i]]
<ipython-input-16-5fee57a14482>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_ids[i] = category_reindex_dict[cat_ids[i]]
<ipython-input-16-5fee57a14482>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_ids[i] = prod_rei

['computers'] 1402


<ipython-input-16-5fee57a14482>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_ids[i] = prod_reindex_dict[prod_ids[i]]
<ipython-input-16-5fee57a14482>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_ids[i] = category_reindex_dict[cat_ids[i]]
<ipython-input-16-5fee57a14482>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_ids[i] = prod_reindex_dict[prod_ids[i]]
<ipython-input-16-5fee57a14482>:10: SettingWithCopyWarning

   product_id          category_id                  category_code   price  \
0     1307073  2053013558920217191             computers.notebook  669.00   
0     1306650  2053013558920217191             computers.notebook  437.57   
0     1307074  2053013558920217191             computers.notebook  772.19   
0     1480163  2053013561092866779              computers.desktop  617.00   
0     1307067  2053013558920217191             computers.notebook  251.74   
..        ...                  ...                            ...     ...   
0     1701076  2053013553031414015  computers.peripherals.monitor  132.54   
0     1700482  2053013553031414015  computers.peripherals.monitor   70.25   
0     9100143  2053013552888807671    computers.peripherals.mouse  122.31   
0     1700690  2053013553031414015  computers.peripherals.monitor  333.29   
0     1307301  2053013558920217191             computers.notebook  475.89   

       brand  user_id  
0       acer     1402  
0       acer     1402  
0  